In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

In [2]:
#####Change the sys_dir into the woring directory where the delivery folder is stored.

sys_dir="/Users/mac/Desktop/Columbia/Spring/Captsone Project/delivery"
data = pd.read_csv(sys_dir+'/data/hurricane_data.csv')
# data = pd.read_csv('/Users/mac/Desktop/Columbia/Spring/Captsone Project/data/hurricane_data.csv')

## Step 1: Create Zone Table Cell by the magnitude of the hurricane data

### Jamaican Zone (center: y=18, x=-77.5)

1) Zone Domain: x=[-80,-74], y=[15,21]


In [3]:
def createZoneTable(i, j,westlimit, southlimit, eastlimit, northlimit):
    zone_table = list()
    long_bucket = (eastlimit-westlimit)/i
    #print (long_bucket)
    lat_bucket = (northlimit-southlimit)/j
    #print (lat_bucket)
    for x in range(1,i+1):
        for y in range(1,j+1):
            swcord = [westlimit+(x-1)*long_bucket, southlimit+(y-1)*lat_bucket]
            secord = [westlimit+x*long_bucket, southlimit+(y-1)*lat_bucket, ]
            necord = [westlimit+x*long_bucket, southlimit+y*lat_bucket,]
            nwcord = [westlimit+(x-1)*long_bucket, southlimit+y*lat_bucket,]
            #print (x,y,(x-1)*j+y,swcord, secord, necord, nwcord, swcord)
            zone_table.append((str(x),str(y),(x-1)*j+y,[swcord, secord, necord, nwcord, swcord]))
    return zone_table


In [4]:
def createGeoJsonObject(zone_table):
    zone_data_dict = dict()
    zone_data_dict['type'] = 'FeatureCollection'
    zone_data_dict_features = list()
    zone_data_dict['features'] = zone_data_dict_features
    
    for i in range(len(zone_table)):
        geometry={}
        coordinates={}
        result={}
        geometry['coordinates']=[zone_table[i][3]]
        geometry['type']='Polygon'
        result['geometry']=geometry
        result['properties']={'zone_x':zone_table[i][0],'zone_y':zone_table[i][1], 'zone_id':zone_table[i][2]}
        result['type']='Feature'
        zone_data_dict_features.append(result)
    return zone_data_dict


## Step 2: Create Zone Table
### Change i, j in the 5th command window to tune the domain size.

In [5]:
##########Setting parameters to create a i*j Map Cells
## i = number of cells on the longitude
## j = number of cells on the latitude


i=6
j=6

#Set the x_center and y_center: the center coordinate of the affected domain
x_center=-77
y_center=18

In [7]:
#createZoneTable(i,j) returns a i*j matrix zone table
westlimit=x_center-i/2
eastlimit=x_center+i/2
southlimit=y_center-j/2
northlimit=y_center+j/2

zone_table  = createZoneTable(i, j,westlimit, southlimit, eastlimit, northlimit)



In [8]:
geojson_obj=createGeoJsonObject(zone_table)

In [9]:
# #check geojson object by looking at the last cell
# # last cell should be the north-east corner cell
# print (geojson_obj["features"][0])
# print (geojson_obj["features"][-1])

In [10]:
from pandas.io.json import json_normalize
df = json_normalize(geojson_obj["features"])
df=df[['geometry.coordinates','properties.zone_id']]
# df.head()

## Step 3: Eliminate Tracks outside of the Expected Map Zone

1) Create binary variable for x and y in the map zone
   
   x_in_zone: 1 if in zone, 0 if out of zone
   
   y_in_zone: 1 if in zone, 0 if out of zone

2) Create binary variable indicating track point in zone:
   
   If x_in_zone+y_in_zone==2: track_in_zone=1
   
3) If for each track, sum('track_in_zone')==0: drop track





In [11]:
def in_zone(data, westlimit, southlimit, eastlimit, northlimit):
    data['x_in_zone'] = np.where((data['x']>=westlimit) & (data['x']<=eastlimit), 1, 0 )
    data['y_in_zone'] = np.where((data['y']>=southlimit) & (data['y']<=northlimit), 1, 0 )
    
    data['in_zone'] = np.where( data['x_in_zone']+data['y_in_zone']==2, 1, 0 )

    temp=pd.DataFrame(data.groupby('code')['in_zone'].sum())
    temp['track_in_zone']=np.where(temp['in_zone']>0,1,0)
    temp.drop(columns=['in_zone'], inplace=True)
    data_jamaica=pd.merge(data, temp, how="inner", on='code', sort=False)
    data_jamaica.drop(columns = ['Unnamed: 0','in_zone'], inplace=True)
    
    data_jamaica.drop(data_jamaica[data_jamaica['track_in_zone']==0].index, axis=0, inplace=True)
    
    return data_jamaica

In [12]:
data_jamaica=in_zone(data,westlimit, southlimit, eastlimit, northlimit)

## Eliminate Track Points outside of the Jamaica Zone

In [13]:
def points_in_zone(data_jamaica):
    data_jamaica['temp']=data_jamaica['x_in_zone']+data_jamaica['y_in_zone']
    data_jamaica_in_zone=data_jamaica.drop(data_jamaica[data_jamaica['temp']<2].index, axis=0)
    data_jamaica_in_zone.drop(columns=['temp','x_in_zone','y_in_zone','track_in_zone'], inplace=True)
    
    return data_jamaica_in_zone

In [14]:
data_jamaica_in_zone = points_in_zone(data_jamaica)

In [15]:
### How many hurricane track in our defined domain
len(data_jamaica_in_zone['code'].unique())

12548

In [16]:
#save hurricane data with zone id and map cell data files
data_jamaica_in_zone.to_csv(sys_dir+'/intermediate/hurricane_in_jamaica.csv')
df.to_csv(sys_dir+'/intermediate/Jamaica_map_zone_id.csv')

## Step 4: Create zone id for hurricane table

 given point (-77.350, 26.146), proportion to cell = (x-westlimit(-120))/bucket, (y-southlimit(-1))/bucket

this point is at 42.65, 27.146 >>> zone_id '4228' (42.65 rounded down, 27.146 roundup)

In [17]:
data=pd.read_csv(sys_dir+'/intermediate/hurricane_in_jamaica.csv', index_col=0)

In [18]:
data.head()

,date,x,y,mslp,mws,rmw,code,mph,cat
272,0001-11-04 00:00,-78.458,16.389,1000.710,47.605,34.482,10,54.782882,0
273,0001-11-04 06:00,-77.079,17.272,999.305,49.447,36.641,10,56.902619,0
274,0001-11-04 12:00,-75.533,18.050,998.110,51.005,37.482,10,58.695534,0
275,0001-11-04 18:00,-74.275,18.727,997.159,54.144,38.041,10,62.307832,0
373,0002-08-15 18:00,-74.382,17.631,923.150,131.924,21.787,13,151.815501,4


In [19]:
# assign zone_id to points data['x','y']
data['zone_id_x'] = data['x'].apply(lambda x: (x-westlimit)).apply(np.ceil)
data['zone_id_y'] = data['y'].apply(lambda x: (x-southlimit)).apply(np.ceil)

data.loc[data.zone_id_x==0, 'zone_id_x'] = 1
data.loc[data.zone_id_y==0, 'zone_id_y'] = 1 

In [20]:
#check if zone_id for x and y coordinates are correct
print (data['zone_id_x'].max(),data['zone_id_y'].max())
print (data['zone_id_x'].min(),data['zone_id_y'].min())

6.0 6.0
1.0 1.0


In [21]:
def zone_id(df):
    step=df['zone_id_x'].max()-df['zone_id_x'].min()+1
    return (df['zone_id_x']-1)*step+df['zone_id_y']

In [22]:
data['zone_id']=zone_id(data)

In [23]:
data['zone_id_x']=data['zone_id_x'].astype(int)
data['zone_id_y']=data['zone_id_y'].astype(int)
data['zone_id']=data['zone_id'].astype(int)

In [24]:
data.drop(columns=['mslp','mws','rmw','mph'],inplace=True)

## Step 5: Import Actual Loss Table and merge with Hurricane Dataset


In [25]:
loss_df=pd.read_csv(sys_dir+"/data/Loss_table_022020.csv")

In [26]:
loss_df.head()

,Code,Loss
0,0,611
1,1,0
2,2,0
3,3,0
4,4,0


In [27]:
loss_df.rename(columns = {'Code':'code', 'Loss':'loss'}, inplace = True) 

In [28]:
data_loss = pd.merge(data, loss_df, how="inner", on='code', sort=False)

In [29]:
data_loss.to_csv(sys_dir+'/intermediate/hurricane_with_loss.csv')

## Step 6: Generate binary variables to mark not neiboring points

In [30]:
data=pd.read_csv(sys_dir+'/intermediate/hurricane_with_loss.csv',index_col=0)

In [31]:
def gen_diff(data):
    data['x_diff0']=data['zone_id_x'].diff()
    data['y_diff0']=data['zone_id_y'].diff()
    data['code_diff']=data['code'].diff()
    
    data=data.fillna(0)
    
    data.loc[data['code_diff']==0.0, 'x_diff'] = data['x_diff0'] 
    data.loc[data['code_diff']==0.0, 'y_diff'] = data['y_diff0'] 
    data=data.fillna(0)
    
    data['x_diff']=data['x_diff'].abs()
    data['y_diff']=data['y_diff'].abs()
    
    data=data.drop(columns=['x_diff0','y_diff0'])
    
    return data

In [32]:
### generate binary variable indicating neighboring points as 1, not neighboring as 0

def gen_neighbor(data):
    data.loc[data['x_diff']+data['y_diff']>1, 'neighbor'] = 0
    data.loc[data['x_diff']+data['y_diff']<=1, 'neighbor'] = 1
    return data

In [33]:
data=gen_diff(data)
data=gen_neighbor(data)

## Step 7: Fulfill missing points between non-adjacent points

In [34]:
def zone_id(df):
    step=df['zone_id_x'].max()-df['zone_id_x'].min()+1
    return (df['zone_id_x']-1)*step+df['zone_id_y']

In [35]:
def gen_id(data):
    
    data.fillna(0)

    data['zone_id_x'] = data['x'].apply(lambda x: (x+81)).apply(np.floor)
    data['zone_id_y'] = data['y'].apply(lambda x: (x-14)).apply(np.floor)

    data.loc[data.zone_id_x==7, 'zone_id_x'] = 6 
    data.loc[data.zone_id_y==7, 'zone_id_y'] = 6 

    data['zone_id']=zone_id(data)
    
    
    return data

In [36]:
def gen_points(x1,y1,x0,y0,cat0,loss0,code0):
    import math
    x_max=max(x1,x0)
    x_min=min(x1,x0)
    x_list=list(range(math.ceil(x_min),math.ceil(x_max)))
    point_x=[x+0.00001 for x in x_list]+[x-0.00001 for x in x_list]
    
    x_y=[(x-x0)*(y1-y0)/(x1-x0)+y0 for x in point_x]
    #print (x_y)
    
    
    #y integers
    y_max=max(y1,y0)
    y_min=min(y1,y0)
    y_list=list(range(math.ceil(y_min),math.ceil(y_max)))
    point_y=[y+0.00001 for y in y_list]+[y-0.00001 for y in y_list]
    #points 
    
    y_x=[(y-y0)*(x1-x0)/(y1-y0)+x0 for y in point_y]
    #print (y_x)
    
    point_x+=y_x
    x_y+=point_y
    
    cat=[cat0]*len(point_x)
    loss=[loss0]*len(point_x)
    code=[code0]*len(point_x)
    neighbor=[1.0]*len(point_x)

    df=pd.DataFrame(list(zip(code,point_x, x_y,cat,neighbor,loss)), columns =['code','x','y','cat','neighbor','loss'])

    return df

# Final Step
## Step 8: Generate A Matrix

In [37]:
### Set the "zone_len" parameter according to our domain size
### zone_len = i*j

zone_len=6*6

In [39]:
#generate A Matrix

a_mat = pd.DataFrame()

for j in list(data['code'].unique()):
    print (j)
    df_track=data[data['code']==j]
    df_track=df_track.reset_index()
    for i in list(df_track.index):
        if df_track['neighbor'][i]==0.0:
            df=gen_points(df_track['x'][i], df_track['y'][i],df_track['x'][i-1], df_track['y'][i-1],df_track['cat'][i-1],df_track['loss'][i-1],df_track['code'][i-1])
            df_track=df_track.append(df).reset_index().drop(columns=['level_0'])

    df_track=gen_id(df_track)    

    track= [0]*zone_len*6+[0]
    passed_cells=df_track["zone_id"].unique()
    #print (df_track["zone_id"].unique())
    for cells in passed_cells:
        max_cat=int(df_track[df_track['zone_id']==cells]['cat'].nlargest(1).values)
        #print (cells, max_cat)
        track[int(cells-1)*6+max_cat-1]=1
    track[-1]=int(df_track[df_track['zone_id']==cells]['loss'].nlargest(1).values)
    a_mat = a_mat.append([track],ignore_index = True)

In [ ]:
a_mat.to_csv(sys_dir+'/output/a_mat_6_6.csv')